**Libraries**

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import sempy.fabric as fabric

**Define a logging dataframe**

In [ ]:
dfLogging = pd.DataFrame(columns = ['LoadId','NotebookId', 'NotebookName', 'WorkspaceId', 'CellId', 'Timestamp', 'ElapsedTime', 'Message', 'ErrorMessage'])
vContext = mssparkutils.runtime.context
vNotebookId = vContext["currentNotebookId"]
vLogNotebookName = vContext["currentNotebookName"]
vWorkspaceId = vContext["currentWorkspaceId"] # where the notebook is running, to not confuse with source and target workspaces

**Parameters --> convert to code for debugging the notebook. otherwise, keep commented as parameters are passed from DevOps pipelines**

pSourceWorkspaceId = ""
pTargetWorkspaceId = ""
pTargetStage = "Stage1"
pDebugMode = "yes"
pTimeoutPerCellInSeconds = "600"
pTimeoutInSeconds = "900"
pProjectName = "fabric-cicd"
pFeatureBranch = "NA"
pMappingConnections = ''

**Check if the source workspace passed from DevOps equals the feature branch name**
- This is a specific handling when a PR is done from the feature branch

In [ ]:
if pSourceWorkspaceId == pFeatureBranch:
    pSourceWorkspaceId = fabric.resolve_workspace_id(workspace=pFeatureBranch)

**Define the DAG**

In [ ]:
dagList = []

# add to the DAG list nb_cicd_post_update_data_pipelines
dagList.append({
            "name": "nb_cicd_post_update_data_pipelines",
            "path": "nb_cicd_post_update_data_pipelines",
            "timeoutPerCellInSeconds": int(pTimeoutPerCellInSeconds),
            "args": {
                "useRootDefaultLakehouse": True,
                "pSourceWorkspaceId":pSourceWorkspaceId,
                "pTargetWorkspaceId":pTargetWorkspaceId,
                "pTargetStage":pTargetStage,
                "pDebugMode":pDebugMode,
                "pProjectName":pProjectName,
                "pMappingConnections": pMappingConnections
                }
        })

# add to the DAG list nb_cicd_post_update_notebooks
dagList.append({
            "name": "nb_cicd_post_update_notebooks",
            "path": "nb_cicd_post_update_notebooks",
            "timeoutPerCellInSeconds": int(pTimeoutPerCellInSeconds),
            "args": {
                "useRootDefaultLakehouse": True,
                "pSourceWorkspaceId":pSourceWorkspaceId,
                "pTargetWorkspaceId":pTargetWorkspaceId,
                "pDebugMode":pDebugMode
                }
        })

# add to the DAG list nb_cicd_post_update_semantic_models
dagList.append({
            "name": "nb_cicd_post_update_semantic_models",
            "path": "nb_cicd_post_update_semantic_models",
            "timeoutPerCellInSeconds": int(pTimeoutPerCellInSeconds),
            "args": {
                "useRootDefaultLakehouse": True,
                "pSourceWorkspaceId":pSourceWorkspaceId,
                "pTargetWorkspaceId":pTargetWorkspaceId,
                "pTargetStage":pTargetStage,
                "pDebugMode":pDebugMode,
                "pProjectName":pProjectName,
                "pMappingConnections": pMappingConnections
                }
        })

DAG = { "activities": dagList,"concurrency": 1, "timeoutInSeconds": int(pTimeoutInSeconds) }


**Run multiple**

In [ ]:
try:
    mssparkutils.notebook.runMultiple(DAG, {"displayDAGViaGraphviz": True})

    # logging
    vMessage = f"succeeded"
    dfLogging.loc[len(dfLogging.index)] = [None, vNotebookId, vLogNotebookName, vWorkspaceId, 'running the DAG', datetime.now(), None, vMessage, ''] 

except Exception as e:
    vMessage = f"failed"
    dfLogging.loc[len(dfLogging.index)] = [None, vNotebookId, vLogNotebookName, vWorkspaceId, 'running the DAG', datetime.now(), None, vMessage, str(e)]
    if pDebugMode == "yes":
        print(str(e))

**Logging**

In [ ]:
try:
    # perform the conversion of columns
    dfLogging = dfLogging.astype({
            "LoadId": "string",	
            "NotebookId": "string", 	
            "NotebookName": "string", 
            "WorkspaceId": "string", 
            "CellId": "string", 
            "Timestamp": "datetime64[ns]", 
            "ElapsedTime": "string", 
            "Message": "string", 
            "ErrorMessage" : "string"
        })

    # save panda dataframe to a spark dataframe 
    sparkDF_Logging = spark.createDataFrame(dfLogging) 

    # save to the lakehouse
    sparkDF_Logging.write.mode("append").format("delta").option("mergeSchema", "true").saveAsTable("staging.notebook_logging")

except Exception as e:
    vMessage = "saving logs to the lakehouse failed"
    if pDebugMode == "yes":
        print(str(e))

**Exit notebook**

In [ ]:
mssparkutils.notebook.exit(f"Notebook <{vLogNotebookName}> run successfully. Check logging table in CI/CD lakehouse for more details.")